In [35]:
import json
import os
import re

import pandas as pd
from python_pdb.aligners import align_sequences
from python_pdb.entities import Structure
from python_pdb.parsers import parse_pdb
from python_pdb.formats.residue import THREE_TO_ONE_CODE

In [2]:
STCRDAB_PATH = '/project/koohylab/shared/tcr_data/raw_DONOTMODIFY/structure/STCRDab_all_2022-11-10/'

## 1. Select TCRs and TCR-pMHCS from the STCRDab

In [3]:
stcrdab_summary = pd.read_csv(os.path.join(STCRDAB_PATH, 'db_summary.dat'), delimiter='\t')
selected_stcrdab = stcrdab_summary.copy()

# Resolution better than 3.50 Å
selected_stcrdab['resolution'] = pd.to_numeric(selected_stcrdab['resolution'], errors='coerce')
selected_stcrdab = selected_stcrdab.query("resolution < 3.50")

# alpha-beta TCRs
selected_stcrdab = selected_stcrdab.query("TCRtype == 'abTCR'")

# MHC class I bound or unbound
selected_stcrdab = selected_stcrdab.query("(mhc_type == 'MH1') | (mhc_type.isnull() & antigen_type.isnull())")

# peptide antigen
selected_stcrdab = selected_stcrdab.query("antigen_type == 'peptide' or antigen_type.isnull()")

# General clean: drop columns that don't contain anything useful
selected_stcrdab = selected_stcrdab.loc[:, selected_stcrdab.nunique() > 1]
selected_stcrdab = selected_stcrdab.dropna(axis=1, how='all')

# Reset Index
selected_stcrdab = selected_stcrdab.reset_index(drop=True)

selected_stcrdab

,pdb,Bchain,Achain,antigen_chain,antigen_name,mhc_chain1,mhc_chain2,docking_angle,beta_subgroup,alpha_subgroup,...,alpha_organism,antigen_organism,mhc_chain1_organism,mhc_chain2_organism,authors,resolution,method,r_free,r_factor,engineered
0,7rk7,E,D,C,tyrosinase peptide,A,B,81.339,TRBV10,TRAV4,...,homo sapiens,homo sapiens,homo sapiens,homo sapiens,"Singh, N.K., Davancaze, L.M., Arbuiso, A., Wei...",2.54,X-RAY DIFFRACTION,0.255,0.211,True
1,7s8i,B,A,NaN,NaN,NaN,NaN,NaN,TRBV27,TRAV27,...,homo sapiens,NaN,NaN,NaN,"Patskovsky, Y., Nyovanie, S., Patskovska, L., ...",1.66,X-RAY DIFFRACTION,0.216,0.167,True
2,7s8j,B,A,NaN,NaN,NaN,NaN,NaN,TRBV27,TRAV27,...,homo sapiens,NaN,NaN,NaN,"Patskovska, L., Patskovsky, Y., Nyovanie, S., ...",1.92,X-RAY DIFFRACTION,0.213,0.168,True
3,2ak4,E,D,C,ebv peptide lpeplpqgqltay,A,B,71.108,TRBV6,TRAV19,...,homo sapiens,NaN,homo sapiens,homo sapiens,"Tynan, F.E., Burrows, S.R., Buckle, A.M., Clem...",2.50,X-RAY DIFFRACTION,0.278,0.246,True
4,7nme,E,D,C,gln-leu-pro-arg-leu-phe-pro-leu-leu,A,B,36.100,TRBV7,TRAV5,...,homo sapiens,homo sapiens,homo sapiens,homo sapiens,"Rizkallah, P.J., Sewell, A.K., Cole, D.K., Wal...",2.20,X-RAY DIFFRACTION,0.274,0.215,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
444,6fr5,B,A,NaN,NaN,NaN,NaN,NaN,TRBV5,TRAV21,...,homo sapiens,NaN,NaN,NaN,"Rizkallah, P.J., Cole, D.K.",1.37,X-RAY DIFFRACTION,0.191,0.168,True
445,5men,E,D,C,ile-leu-ala-lys-phe-leu-his-trp-leu,A,B,40.823,TRBV6,TRAV22,...,homo sapiens,homo sapiens,homo sapiens,homo sapiens,"Rizkallah, P.J., Lloyd, A., Crowther, M., Cole...",2.81,X-RAY DIFFRACTION,0.272,0.189,True
446,3vxu,J,I,H,10-mer peptide from protein nef,F,G,NaN,TRBV27,TRAV12,...,homo sapiens,human immunodeficiency virus 1,homo sapiens,homo sapiens,"Shimizu, A., Fukai, S., Yamagata, A., Iwamoto, A.",2.70,X-RAY DIFFRACTION,0.323,0.272,True
447,6fup,B,A,NaN,NaN,NaN,NaN,NaN,TRBV24,TRAV8,...,homo sapiens,NaN,NaN,NaN,"Rizkallah, P.J., Cole, D.K.",1.72,X-RAY DIFFRACTION,0.215,0.176,True


In [4]:
selected_stcrdab['file_path_imgt'] = selected_stcrdab['pdb'].map(lambda pdb_id: os.path.join(STCRDAB_PATH, 'imgt', pdb_id + '.pdb'))
selected_stcrdab['file_path_raw'] = selected_stcrdab['pdb'].map(lambda pdb_id: os.path.join(STCRDAB_PATH, 'raw', pdb_id + '.pdb'))

In [5]:
selected_stcrdab['chains'] = selected_stcrdab[['Achain', 'Bchain', 'antigen_chain', 'mhc_chain1']].apply(lambda chains: list(''.join(chains.dropna())), axis=1)

## 2. Disgard structures missing residues in important domains

These domains include:

* TCR variable domain
* peptide

In [6]:
def get_header(pdb_contents):
    '''Get the header lines from the contents of a pdb file.'''
    header = []
    for line in pdb_contents.split('\n'):
        record_type = line[0:6]
        if record_type in ("ATOM  ", "HETATM", "MODEL "):
            break

        header.append(line)

    return '\n'.join(header)


def get_missing_residues(header):
    '''
    Extract all missing residues from the header of a pdb file. Returns residue name, chain id, residue/sequence id,
    and insert id if available.
    '''
    lines = re.findall(r'^REMARK 465\s+\w?\s+(\w{3}) (\w)\s+(\d+)(\w)?', header, flags=re.MULTILINE)
    return [{'residue_name': res_name, 'chain_id': chain, 'residue_seq_id': int(seq_id), 'residue_insert_code': insert_id}
            for res_name, chain, seq_id, insert_id in lines]


def get_missing_atoms(header: str) -> list[dict]:
    '''Extract residues with missing atoms from the header of a pdb file.'''
    lines = re.findall(r'^REMARK 470\s+\w?\s+(\w{3}) (\w)\s+(\d+)(\w)?', header, flags=re.MULTILINE)
    return [{'residue_name': res_name, 'chain_id': chain, 'residue_seq_id': int(seq_id), 'residue_insert_code': insert_id}
            for res_name, chain, seq_id, insert_id in lines]

raw_structure_dfs = {}

for _, pdb_id, path in selected_stcrdab[['pdb', 'file_path_raw']].drop_duplicates().itertuples():
    with open(path, 'r') as fh:
        pdb_contents = fh.read()
        
    structure = parse_pdb(pdb_contents, silent=True).to_pandas()
    header = get_header(pdb_contents)
    
    structure['missing'] = False
    
    missing_residues = pd.DataFrame(get_missing_residues(header))
    missing_residues['missing'] = True
    
    residues_missing_atoms = pd.DataFrame(get_missing_atoms(header))
    residues_missing_atoms['missing'] = True
    
    missing = pd.concat([missing_residues, residues_missing_atoms]).reset_index(drop=True)
    
    for _, row in missing.iterrows():
        chain_before = structure.query("chain_id == @row.chain_id and residue_seq_id <= @row.residue_seq_id")
        chain_after = structure.query("chain_id == @row.chain_id and residue_seq_id > @row.residue_seq_id")

        new_chain = pd.concat([chain_before, row.to_frame().T, chain_after])
        structure = pd.concat([structure.query('chain_id != @row.chain_id'), new_chain]).reset_index(drop=True)
    
    raw_structure_dfs[pdb_id] = structure

In [7]:
def screen_variable(chain, raw_chain):
    def get_sequence(df):
        return ''.join(df['residue_name'].map(lambda tlc: THREE_TO_ONE_CODE[tlc]).to_list())
    
    raw_seq = get_sequence(raw_chain)
    seq = get_sequence(chain)
    alignment, _ = align_sequences(raw_seq, seq)
    
    raw_index = 0
    index = 0
    current_seq_id = 0

    for raw_res, res in alignment:
        if raw_res == '-':
            index += 1
            continue
        
        if res != '-':
            current_seq_id = chain.iloc[index]['residue_seq_id']
            
        if raw_chain.iloc[raw_index]['missing'] and current_seq_id < 128:
            return False

        if res == '-':
            raw_index += 1
            continue

        index +=1
        raw_index += 1
    
    return True
    
selected_entries = []
for _, entry in selected_stcrdab.iterrows():
    print(entry.pdb)
    
    # 1. check if there are any missing items
    raw_structure = raw_structure_dfs[entry.pdb]
    
    if len(raw_structure.query('missing == True')) == 0:
        print('all clear, adding to selection')
        selected_entries.append(entry.to_frame().T)
        continue
    
    with open(entry['file_path_imgt'], 'r') as fh:
        structure = parse_pdb(fh.read(), silent=True).to_pandas()

    # 2. look if they are in TCR variable domains
    # 2a. alpha chain
    print('looking at alpha chain')
    alpha = (structure.query("record_type == 'ATOM' and chain_id == @entry.Achain")
                      .drop_duplicates(['chain_id', 'residue_seq_id', 'residue_insert_code']).reset_index())
    
    raw_alpha = (raw_structure.query("record_type == 'ATOM' and chain_id == @entry.Achain")
                              .drop_duplicates(['chain_id', 'residue_seq_id', 'residue_insert_code']).reset_index())
    
    if not screen_variable(alpha, raw_alpha):
        print('check failed, missing residue found')
        continue
    
    # 2b. beta chain
    print('looking at beta chain')
    beta = (structure.query("record_type == 'ATOM' and chain_id == @entry.Bchain")
                     .drop_duplicates(['chain_id', 'residue_seq_id', 'residue_insert_code']).reset_index())
    
    raw_beta = (raw_structure.query("record_type == 'ATOM' and chain_id == @entry.Bchain")
                             .drop_duplicates(['chain_id', 'residue_seq_id', 'residue_insert_code']).reset_index())
    
    if not screen_variable(beta, raw_beta):
        print('check failed, missing residue found')
        continue
    
    # 3. look if they are in the peptide
    print('looking at peptide')
    raw_peptide = raw_structure.query('chain_id == @entry.antigen_chain')
    
    if len(raw_peptide.query("missing == True")) > 0:
        print('check failed, missing residue found')
        continue
    
    # Structure passed all checks so keep it
    print('all clear, adding to selection')
    selected_entries.append(entry.to_frame().T)

new_df = pd.concat(selected_entries)

7rk7
looking at alpha chain
looking at beta chain
looking at peptide
all clear, adding to selection
7s8i
looking at alpha chain
looking at beta chain
looking at peptide
all clear, adding to selection
7s8j
looking at alpha chain
looking at beta chain
looking at peptide
all clear, adding to selection
2ak4
looking at alpha chain
looking at beta chain
looking at peptide
all clear, adding to selection
7nme
looking at alpha chain
looking at beta chain
looking at peptide
all clear, adding to selection
7pbe
looking at alpha chain
looking at beta chain
looking at peptide
all clear, adding to selection
5wkh
looking at alpha chain
looking at beta chain
looking at peptide
all clear, adding to selection
6fr7
looking at alpha chain
looking at beta chain
looking at peptide
all clear, adding to selection
3w0w
looking at alpha chain
looking at beta chain
looking at peptide
all clear, adding to selection
5w1v
looking at alpha chain
looking at beta chain
looking at peptide
all clear, adding to selection


looking at beta chain
looking at peptide
all clear, adding to selection
5bs0
all clear, adding to selection
5m00
looking at alpha chain
looking at beta chain
looking at peptide
all clear, adding to selection
6fuo
all clear, adding to selection
7rm4
looking at alpha chain
looking at beta chain
looking at peptide
all clear, adding to selection
6eqb
all clear, adding to selection
7r7z
looking at alpha chain
looking at beta chain
looking at peptide
all clear, adding to selection
3dx9
all clear, adding to selection
6l9l
all clear, adding to selection
6uk4
looking at alpha chain
looking at beta chain
looking at peptide
all clear, adding to selection
7ea6
looking at alpha chain
looking at beta chain
looking at peptide
all clear, adding to selection
2vlm
looking at alpha chain
looking at beta chain
looking at peptide
all clear, adding to selection
5tje
looking at alpha chain
looking at beta chain
looking at peptide
all clear, adding to selection
6dfq
looking at alpha chain
looking at beta chai

looking at beta chain
looking at peptide
all clear, adding to selection
7na5
looking at alpha chain
looking at beta chain
looking at peptide
all clear, adding to selection
5c07
looking at alpha chain
looking at beta chain
looking at peptide
all clear, adding to selection
4pri
all clear, adding to selection
7dzm
looking at alpha chain
looking at beta chain
looking at peptide
all clear, adding to selection
5m02
looking at alpha chain
looking at beta chain
looking at peptide
all clear, adding to selection
3axl
looking at alpha chain
looking at beta chain
looking at peptide
all clear, adding to selection
2ckb
all clear, adding to selection
3tfk
looking at alpha chain
looking at beta chain
looking at peptide
all clear, adding to selection
3sjv
looking at alpha chain
looking at beta chain
looking at peptide
all clear, adding to selection
3axl
looking at alpha chain
looking at beta chain
looking at peptide
all clear, adding to selection
8d5p
looking at alpha chain
looking at beta chain
lookin

looking at beta chain
looking at peptide
all clear, adding to selection
3qjf
looking at alpha chain
looking at beta chain
looking at peptide
all clear, adding to selection
4jfh
all clear, adding to selection
4onh
looking at alpha chain
looking at beta chain
looking at peptide
all clear, adding to selection
7n1d
looking at alpha chain
looking at beta chain
looking at peptide
all clear, adding to selection
5yxu
looking at alpha chain
looking at beta chain
looking at peptide
all clear, adding to selection
6fra
all clear, adding to selection
2nx5
looking at alpha chain
looking at beta chain
looking at peptide
all clear, adding to selection
4l3e
all clear, adding to selection
3o4l
all clear, adding to selection
2pyf
all clear, adding to selection
5nme
looking at alpha chain
looking at beta chain
looking at peptide
all clear, adding to selection
1kgc
looking at alpha chain
looking at beta chain
looking at peptide
all clear, adding to selection
5d2n
looking at alpha chain
looking at beta chai

looking at alpha chain
looking at beta chain
looking at peptide
all clear, adding to selection
7pb2
looking at alpha chain
looking at beta chain
looking at peptide
all clear, adding to selection
2nx5
looking at alpha chain
looking at beta chain
looking at peptide
all clear, adding to selection
6dfq
looking at alpha chain
looking at beta chain
looking at peptide
all clear, adding to selection
7ea6
looking at alpha chain
looking at beta chain
looking at peptide
all clear, adding to selection
2ckb
all clear, adding to selection
6vrn
looking at alpha chain
looking at beta chain
looking at peptide
all clear, adding to selection
2esv
looking at alpha chain
looking at beta chain
looking at peptide
all clear, adding to selection
6fr8
looking at alpha chain
looking at beta chain
looking at peptide
all clear, adding to selection
4grm
looking at alpha chain
looking at beta chain
looking at peptide
all clear, adding to selection
5hhm
looking at alpha chain
looking at beta chain
looking at peptide


In [8]:
selected_strcdab = new_df

## 3. Annotate the state of the TCR: unbound (*apo*) or bound (*holo*)

In [9]:
selected_stcrdab['state'] = selected_stcrdab.apply(lambda row: 'apo' if pd.isna(row.antigen_chain) and pd.isna(row.mhc_chain1) else 'holo', axis=1)

## 4. Drop duplicate PDB IDs

Making sure not to select unbound example from PDB ID with bound TCR-pMHC

In [10]:
holo_pdb_ids = selected_stcrdab.query("state == 'holo'")['pdb'].unique().tolist()
selected_stcrdab = selected_stcrdab.query("state == 'holo' or (state == 'apo' and pdb not in @holo_pdb_ids)")
selected_stcrdab = selected_stcrdab.drop_duplicates('pdb')
selected_stcrdab

,pdb,Bchain,Achain,antigen_chain,antigen_name,mhc_chain1,mhc_chain2,docking_angle,beta_subgroup,alpha_subgroup,...,authors,resolution,method,r_free,r_factor,engineered,file_path_imgt,file_path_raw,chains,state
0,7rk7,E,D,C,tyrosinase peptide,A,B,81.339,TRBV10,TRAV4,...,"Singh, N.K., Davancaze, L.M., Arbuiso, A., Wei...",2.54,X-RAY DIFFRACTION,0.255,0.211,True,/project/koohylab/shared/tcr_data/raw_DONOTMOD...,/project/koohylab/shared/tcr_data/raw_DONOTMOD...,"[D, E, C, A]",holo
1,7s8i,B,A,NaN,NaN,NaN,NaN,NaN,TRBV27,TRAV27,...,"Patskovsky, Y., Nyovanie, S., Patskovska, L., ...",1.66,X-RAY DIFFRACTION,0.216,0.167,True,/project/koohylab/shared/tcr_data/raw_DONOTMOD...,/project/koohylab/shared/tcr_data/raw_DONOTMOD...,"[A, B]",apo
2,7s8j,B,A,NaN,NaN,NaN,NaN,NaN,TRBV27,TRAV27,...,"Patskovska, L., Patskovsky, Y., Nyovanie, S., ...",1.92,X-RAY DIFFRACTION,0.213,0.168,True,/project/koohylab/shared/tcr_data/raw_DONOTMOD...,/project/koohylab/shared/tcr_data/raw_DONOTMOD...,"[A, B]",apo
3,2ak4,E,D,C,ebv peptide lpeplpqgqltay,A,B,71.108,TRBV6,TRAV19,...,"Tynan, F.E., Burrows, S.R., Buckle, A.M., Clem...",2.50,X-RAY DIFFRACTION,0.278,0.246,True,/project/koohylab/shared/tcr_data/raw_DONOTMOD...,/project/koohylab/shared/tcr_data/raw_DONOTMOD...,"[D, E, C, A]",holo
4,7nme,E,D,C,gln-leu-pro-arg-leu-phe-pro-leu-leu,A,B,36.100,TRBV7,TRAV5,...,"Rizkallah, P.J., Sewell, A.K., Cole, D.K., Wal...",2.20,X-RAY DIFFRACTION,0.274,0.215,True,/project/koohylab/shared/tcr_data/raw_DONOTMOD...,/project/koohylab/shared/tcr_data/raw_DONOTMOD...,"[D, E, C, A]",holo
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
443,6q3s,E,D,C,ser-leu-leu-met-trp-ile-thr-gln-val,A,B,67.059,TRBV6,TRAV21,...,"Meijers, R., Anjanappa, R., Springer, S., Garc...",2.50,X-RAY DIFFRACTION,0.273,0.229,True,/project/koohylab/shared/tcr_data/raw_DONOTMOD...,/project/koohylab/shared/tcr_data/raw_DONOTMOD...,"[D, E, C, A]",holo
444,6fr5,B,A,NaN,NaN,NaN,NaN,NaN,TRBV5,TRAV21,...,"Rizkallah, P.J., Cole, D.K.",1.37,X-RAY DIFFRACTION,0.191,0.168,True,/project/koohylab/shared/tcr_data/raw_DONOTMOD...,/project/koohylab/shared/tcr_data/raw_DONOTMOD...,"[A, B]",apo
445,5men,E,D,C,ile-leu-ala-lys-phe-leu-his-trp-leu,A,B,40.823,TRBV6,TRAV22,...,"Rizkallah, P.J., Lloyd, A., Crowther, M., Cole...",2.81,X-RAY DIFFRACTION,0.272,0.189,True,/project/koohylab/shared/tcr_data/raw_DONOTMOD...,/project/koohylab/shared/tcr_data/raw_DONOTMOD...,"[D, E, C, A]",holo
447,6fup,B,A,NaN,NaN,NaN,NaN,NaN,TRBV24,TRAV8,...,"Rizkallah, P.J., Cole, D.K.",1.72,X-RAY DIFFRACTION,0.215,0.176,True,/project/koohylab/shared/tcr_data/raw_DONOTMOD...,/project/koohylab/shared/tcr_data/raw_DONOTMOD...,"[A, B]",apo


## 5. Group TCRs by CDR sequences

In [11]:
IMGT_CDR1 = set(range(27, 38 + 1))
IMGT_CDR2 = set(range(56, 65 + 1))
IMGT_CDR3 = set(range(105, 117 + 1))

def assign_cdr_number(imgt_id: str | int | None) -> int | None:
    '''
    Map imgt_id to CDR domains, return number associated with domain or return None if input is not in a CDR
    domain.
    '''
    if not imgt_id:
        return None

    if type(imgt_id) is not int:
        seq_id = int(''.join([char for char in imgt_id if char.isnumeric()]))

    else:
        seq_id = imgt_id

    if seq_id in IMGT_CDR1:
        return 1

    if seq_id in IMGT_CDR2:
        return 2

    if seq_id in IMGT_CDR3:
        return 3

    return None

cdr_1_alpha_seq = []
cdr_2_alpha_seq = []
cdr_3_alpha_seq = []

cdr_1_beta_seq = []
cdr_2_beta_seq = []
cdr_3_beta_seq = []

peptide_seq = []

mhc_chain_1_seq = []
mhc_chain_2_seq = []


for index, stcrdab_entry in selected_stcrdab.iterrows():
    with open(stcrdab_entry['file_path_imgt'], 'r') as fh:
            structure = parse_pdb(fh.read(), silent=True)
    
    # Pre-processing of structure
    structure.dehydrate()
    
    structure_df = structure.to_pandas()

    structure_df['cdr'] = structure_df['residue_seq_id'].map(lambda id_: assign_cdr_number(str(id_)))
    structure_df['res_olc'] = structure_df['residue_name'].map(lambda res_name: THREE_TO_ONE_CODE[res_name] if res_name in THREE_TO_ONE_CODE else None)

    # Alpha CDRs
    cdr_1_alpha_seq.append(''.join(structure_df.query(f"cdr == 1 & chain_id == '{stcrdab_entry['Achain']}'")
                                               .drop_duplicates(['residue_seq_id', 'residue_insert_code'])['res_olc']
                                               .dropna()))
    cdr_2_alpha_seq.append(''.join(structure_df.query(f"cdr == 2 & chain_id == '{stcrdab_entry['Achain']}'")
                                               .drop_duplicates(['residue_seq_id', 'residue_insert_code'])['res_olc']
                                               .dropna()))
    cdr_3_alpha_seq.append(''.join(structure_df.query(f"cdr == 3 & chain_id == '{stcrdab_entry['Achain']}'")
                                               .drop_duplicates(['residue_seq_id', 'residue_insert_code'])['res_olc']
                                               .dropna()))
    # Beta CDRs
    cdr_1_beta_seq.append(''.join(structure_df.query(f"cdr == 1 & chain_id == '{stcrdab_entry['Bchain']}'")
                                              .drop_duplicates(['residue_seq_id', 'residue_insert_code'])['res_olc']
                                              .dropna()))
    cdr_2_beta_seq.append(''.join(structure_df.query(f"cdr == 2 & chain_id == '{stcrdab_entry['Bchain']}'")
                                              .drop_duplicates(['residue_seq_id', 'residue_insert_code'])['res_olc']
                                              .dropna()))
    cdr_3_beta_seq.append(''.join(structure_df.query(f"cdr == 3 & chain_id == '{stcrdab_entry['Bchain']}'")
                                              .drop_duplicates(['residue_seq_id', 'residue_insert_code'])['res_olc']
                                              .dropna()))
    if stcrdab_entry['state'] == 'holo':
        peptide_seq.append(''.join(structure_df.query(f"chain_id == '{stcrdab_entry['antigen_chain']}'")
                                               .drop_duplicates(['residue_seq_id', 'residue_insert_code'])['res_olc']
                                               .dropna())
             )
        
        mhc_chain_1_seq.append(''.join(structure_df.query(f"chain_id == '{stcrdab_entry['mhc_chain1']}'")
                                                   .drop_duplicates(['residue_seq_id', 'residue_insert_code'])['res_olc']
                                                   .dropna())
             )
    
        mhc_chain_2_seq.append(''.join(structure_df.query(f"chain_id == '{stcrdab_entry['mhc_chain2']}'")
                                                   .drop_duplicates(['residue_seq_id', 'residue_insert_code'])['res_olc']
                                                   .dropna())
         )

    else:
        peptide_seq.append(None)
        mhc_chain_1_seq.append(None)
        mhc_chain_2_seq.append(None)

/home/b/bmcmaste/.local/lib/python3.11/site-packages/python_pdb/entities.py:53: UserWarning: Residue(HOH, 217) not found in Chain(G).
  warnings.warn(f'{child} not found in {self}.')
/home/b/bmcmaste/.local/lib/python3.11/site-packages/python_pdb/entities.py:53: UserWarning: Residue(HOH, 218) not found in Chain(G).
  warnings.warn(f'{child} not found in {self}.')
/home/b/bmcmaste/.local/lib/python3.11/site-packages/python_pdb/entities.py:53: UserWarning: Residue(HOH, 219) not found in Chain(G).
  warnings.warn(f'{child} not found in {self}.')
/home/b/bmcmaste/.local/lib/python3.11/site-packages/python_pdb/entities.py:53: UserWarning: Residue(HOH, 220) not found in Chain(G).
  warnings.warn(f'{child} not found in {self}.')
/home/b/bmcmaste/.local/lib/python3.11/site-packages/python_pdb/entities.py:53: UserWarning: Residue(HOH, 1183) not found in Chain(C).
  warnings.warn(f'{child} not found in {self}.')


In [12]:
selected_stcrdab['cdr_1_alpha_seq'] = cdr_1_alpha_seq
selected_stcrdab['cdr_2_alpha_seq'] = cdr_2_alpha_seq
selected_stcrdab['cdr_3_alpha_seq'] = cdr_3_alpha_seq

selected_stcrdab['cdr_1_beta_seq'] = cdr_1_beta_seq
selected_stcrdab['cdr_2_beta_seq'] = cdr_2_beta_seq
selected_stcrdab['cdr_3_beta_seq'] = cdr_3_beta_seq

selected_stcrdab['peptide_seq'] = peptide_seq

selected_stcrdab['mhc_chain_1_seq'] = mhc_chain_1_seq
selected_stcrdab['mhc_chain_2_seq'] = mhc_chain_2_seq

selected_stcrdab

,pdb,Bchain,Achain,antigen_chain,antigen_name,mhc_chain1,mhc_chain2,docking_angle,beta_subgroup,alpha_subgroup,...,state,cdr_1_alpha_seq,cdr_2_alpha_seq,cdr_3_alpha_seq,cdr_1_beta_seq,cdr_2_beta_seq,cdr_3_beta_seq,peptide_seq,mhc_chain_1_seq,mhc_chain_2_seq
0,7rk7,E,D,C,tyrosinase peptide,A,B,81.339,TRBV10,TRAV4,...,holo,NIATNDY,GYKTK,LVALNYGGSQGNLI,ENHRY,SYGVKD,AISPTEEGGLIFPGNTIY,YMDGTMSQV,GSHSMRYFFTSVSRPGRGEPRFIAVGYVDDTQFVRFDSDAASQRME...,TPKIQVYSRHPANGKFLNCYVSGFHPSDIEVDKNGRIEKVEHSDLS...
1,7s8i,B,A,NaN,NaN,NaN,NaN,NaN,TRBV27,TRAV27,...,apo,SVFSS,VVTGGEV,AGYGGGSNYKLT,MNHEY,SMNVEV,ASRLTGRVHGYT,None,None,None
2,7s8j,B,A,NaN,NaN,NaN,NaN,NaN,TRBV27,TRAV27,...,apo,SVFSS,VVTGGEV,AGYGGGSNYKLT,MNHEY,SMNVEV,ASRLTGRVHGYT,None,None,None
3,2ak4,E,D,C,ebv peptide lpeplpqgqltay,A,B,71.108,TRBV6,TRAV19,...,holo,TRDTTYY,RNSFDEQN,ALSGFYNTDKLI,MNHNS,SASEGT,ASPGLAGEYEQY,LPEPLPQGQLTAY,GSHSMRYFYTAMSRPGRGEPRFIAVGYVDDTQFVRFDSDAASPRTE...,TPKIQVYSRHPAENGKSNFLNCYVSGFHPSDIEVDLLKNGERIEKV...
4,7nme,E,D,C,gln-leu-pro-arg-leu-phe-pro-leu-leu,A,B,36.100,TRBV7,TRAV5,...,holo,DSSSTY,IFSNMDM,AEPSGNTGKLI,SEHNR,FQNEAQ,ASSLHHEQY,QLPRLFPLL,GSHSMRYFSTSVSRPGRGEPRFIAVGYVDDTQFVRFDSDAASQRME...,TPKIQVYSRHPAENGKSNFLNCYVSGFHPSDIEVDLLKNGERIEKV...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
443,6q3s,E,D,C,ser-leu-leu-met-trp-ile-thr-gln-val,A,B,67.059,TRBV6,TRAV21,...,holo,DSAIYN,IQSSQRE,AVRPTSGGSYIPT,MNHEY,SVGAGI,ASSYVGNTGELF,SLLMWITQV,GSHSMRYFFTSVSRPGRGEPRFIAVGYVDDTQFVRFDSDAASQRME...,TPKIQVYSRHPAENGKSNFLNCYVSGFHPSDIEVDLLKNGERIEKV...
444,6fr5,B,A,NaN,NaN,NaN,NaN,NaN,TRBV5,TRAV21,...,apo,DSAIYN,IQSSQRE,AVTNFNKFY,SGHRS,YFSETQ,ASSFDSGNSPLH,None,None,None
445,5men,E,D,C,ile-leu-ala-lys-phe-leu-his-trp-leu,A,B,40.823,TRBV6,TRAV22,...,holo,DSVNN,IPSGT,AVDSATSGTYKYI,MNHEY,SVGAGI,ASSYQGTEAF,ILAKFLHWL,GSHSMRYFFTSVSRPGRGEPRFIAVGYVDDTQFVRFDSDAASQRME...,TPKIQVYSRHPAENGKSNFLNCYVSGFHPSDIEVDLLKNGERIEKV...
447,6fup,B,A,NaN,NaN,NaN,NaN,NaN,TRBV24,TRAV8,...,apo,SSVPPY,YTSAATLV,AVSEQDDKII,KGHDR,SFDVKD,ATSDESYGYT,None,None,None


## 6. Select groups with both *apo* and *holo* confomations

In [13]:
apo_holo_dfs = []

for sequences, tcr_group in selected_stcrdab.groupby(['cdr_1_alpha_seq', 'cdr_2_alpha_seq', 'cdr_3_alpha_seq', 'cdr_1_beta_seq', 'cdr_2_beta_seq', 'cdr_3_beta_seq']):
    # Screen out groups that don't have apo and holo forms
    if 'holo' not in tcr_group['state'].unique().tolist() or 'apo' not in tcr_group['state'].unique().tolist():
        continue
        
    tcr_group = tcr_group.copy()
    tcr_group['cdr_sequence_collated'] = '-'.join(sequences)
    
    apo_holo_dfs.append(tcr_group)

apo_holo_tcrs = pd.concat(apo_holo_dfs).reset_index(drop=True)
apo_holo_tcrs

,pdb,Bchain,Achain,antigen_chain,antigen_name,mhc_chain1,mhc_chain2,docking_angle,beta_subgroup,alpha_subgroup,...,cdr_1_alpha_seq,cdr_2_alpha_seq,cdr_3_alpha_seq,cdr_1_beta_seq,cdr_2_beta_seq,cdr_3_beta_seq,peptide_seq,mhc_chain_1_seq,mhc_chain_2_seq,cdr_sequence_collated
0,6am5,E,D,C,ser-met-leu-gly-ile-gly-ile-val-pro-val,A,B,33.258,TRBV6,TRAV12,...,DRGSQS,IYSNGD,AVNFGGGKLI,MRHNA,SNTAGT,ASSLSFGTEAF,SMLGIGIVPV,GSHSMRYFFTSVSRPGRGEPRFIAVGYVDDTQFVRFDSDAASQRME...,TPKIQVYSRHPAENGKSNFLNCYVSGFHPSDIEVDLLKNGERIEKV...,DRGSQS-IYSNGD-AVNFGGGKLI-MRHNA-SNTAGT-ASSLSFGTEAF
1,3qdj,E,D,C,mart-1(27-35) peptide,A,B,33.874,TRBV6,TRAV12,...,DRGSQS,IYSNGD,AVNFGGGKLI,MRHNA,SNTAGT,ASSLSFGTEAF,AAGIGILTV,GSHSMRYFFTSVSRPGRGEPRFIAVGYVDDTQFVRFDSDAASQRME...,TPKIQVYSRHPAENGKSNFLNCYVSGFHPSDIEVDLLKNGERIEKV...,DRGSQS-IYSNGD-AVNFGGGKLI-MRHNA-SNTAGT-ASSLSFGTEAF
2,3qeu,E,D,NaN,NaN,NaN,NaN,NaN,TRBV6,TRAV12,...,DRGSQS,IYSNGD,AVNFGGGKLI,MRHNA,SNTAGT,ASSLSFGTEAF,None,None,None,DRGSQS-IYSNGD-AVNFGGGKLI-MRHNA-SNTAGT-ASSLSFGTEAF
3,3qdg,E,D,C,mart-1 (26-35) peptide,A,B,34.175,TRBV6,TRAV12,...,DRGSQS,IYSNGD,AVNFGGGKLI,MRHNA,SNTAGT,ASSLSFGTEAF,ELAGIGILTV,GSHSMRYFFTSVSRPGRGEPRFIAVGYVDDTQFVRFDSDAASQRME...,TPKIQVYSRHPAENGKSNFLNCYVSGFHPSDIEVDLLKNGERIEKV...,DRGSQS-IYSNGD-AVNFGGGKLI-MRHNA-SNTAGT-ASSLSFGTEAF
4,6amu,E,D,C,met-met-trp-asp-arg-gly-leu-gly-met-met,A,B,36.974,TRBV6,TRAV12,...,DRGSQS,IYSNGD,AVNFGGGKLI,MRHNA,SNTAGT,ASSLSFGTEAF,MMWDRGLGMM,SHSMRYFFTSVSRPGRGEPRFIAVGYVDDTQFVRFDSDAASQRMEP...,TPKIQVYSRHPAENGKSNFLNCYVSGFHPSDIEVDLLKNGERIEKV...,DRGSQS-IYSNGD-AVNFGGGKLI-MRHNA-SNTAGT-ASSLSFGTEAF
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72,2oi9,C,B,Q,peptide (gln)(leu)(ser)(pro)(phe)(pro)(phe)(as...,A,NaN,48.240,TRBV13,TRAV9,...,YSATPY,YYSGDPVV,AVSGFASALT,NNHNN,SYGAGS,ASGGGGTLY,QLSPFPFDL,GPHSMRYYETATSRRGLGEPRYTSVGYVDDKEFVRFDSDAENPRYE...,,YSATPY-YYSGDPVV-AVSGFASALT-NNHNN-SYGAGS-ASGGGGTLY
73,1mwa,B,A,P,dev8,H,L,22.774,TRBV13,TRAV9,...,YSATPY,YYSGDPVV,AVSGFASALT,NNHNN,SYGAGS,ASGGGGTLY,EQYKFYSV,GPHSLRYFVTAVSRPGLGEPRYMEVGYVDDTEFVRFDSDAENPRYE...,TPQIQVYSRHPPENGKPNILNCYVTQFHPPHIEIQMLKNGKKIPKV...,YSATPY-YYSGDPVV-AVSGFASALT-NNHNN-SYGAGS-ASGGGGTLY
74,1tcr,B,A,NaN,NaN,NaN,NaN,NaN,TRBV13,TRAV9,...,YSATPY,YYSGDPVV,AVSGFASALT,NNHNN,SYGAGS,ASGGGGTLY,None,None,None,YSATPY-YYSGDPVV-AVSGFASALT-NNHNN-SYGAGS-ASGGGGTLY
75,7n1e,E,D,C,spike protein s2,A,B,35.924,TRBV11,TRAV16,...,YSGSPE,HISR,ALSGFNNAGNMLT,SGHAT,FQNNGV,ASSLGGAGGADTQY,RLQSLQTYV,GSHSMRYFFTSVSRPGRGEPRFIAVGYVDDTQFVRFDSDAASQRME...,TPKIQVYSRHPAENGKSNFLNCYVSGFHPSDIEVDLLKNGERIEKV...,YSGSPE-HISR-ALSGFNNAGNMLT-SGHAT-FQNNGV-ASSLGGA...


## 7. Manually check Apo files

In [15]:
apo_tcrs = apo_holo_tcrs.query("state == 'apo'")[['pdb', 'Achain', 'Bchain', 'file_path_imgt']]
apo_tcrs

,pdb,Achain,Bchain,file_path_imgt
2,3qeu,D,E,/project/koohylab/shared/tcr_data/raw_DONOTMOD...
7,7n1d,A,B,/project/koohylab/shared/tcr_data/raw_DONOTMOD...
9,5nmd,A,B,/project/koohylab/shared/tcr_data/raw_DONOTMOD...
14,7amp,A,B,/project/koohylab/shared/tcr_data/raw_DONOTMOD...
24,3vxt,A,B,/project/koohylab/shared/tcr_data/raw_DONOTMOD...
25,7r7z,A,B,/project/koohylab/shared/tcr_data/raw_DONOTMOD...
27,3vxq,D,E,/project/koohylab/shared/tcr_data/raw_DONOTMOD...
30,2pyf,A,B,/project/koohylab/shared/tcr_data/raw_DONOTMOD...
35,2bnu,A,B,/project/koohylab/shared/tcr_data/raw_DONOTMOD...
39,4jfh,D,E,/project/koohylab/shared/tcr_data/raw_DONOTMOD...


In [16]:
acceptable = [
True,  # 2   3qeu  D   E   a bit suspect because it is close to the other TCR in the file but will be allowed
True,  # 7   7n1d  A   B
True,  # 9   5nmd  A   B
True,  # 14  7amp  A   B
True,  # 24  3vxt  A   B
True,  # 25  7r7z  A   B
True,  # 27  3vxq  D   E
True,  # 30  2pyf  A   B
True,  # 35  2bnu  A   B
True,  # 39  4jfh  D   E
True,  # 44  6at6  A   B
True,  # 45  5iw1  A   B
True,  # 53  3utp  D   E weird helice annotated in CDR loop by pymol but not contacting anything
True,  # 57  6vth  D   E
False, # 60  3skn  A   B seems too close to the other TCRs and loops almost look like they are being pushed away
True,  # 61  2vlm  D   E
True,  # 69  1kgc  D   E
True,  # 74  1tcr  A   B there is a sugar or something quite close to the loops but does not seem to be in contact
True,  # 76  7n1c  D   E
]

In [29]:
apo_holo_tcrs['valid'] = True
apo_holo_tcrs.loc[apo_holo_tcrs['state'] == 'apo', 'valid'] = acceptable

re-screen for apo-holo tcrs after dropping non-valid holos...

In [31]:
apo_holo_dfs = []

for group_name, tcr_group in apo_holo_tcrs.query("valid == True").groupby('cdr_sequence_collated'):
    # Screen out groups that don't have apo and holo forms
    if 'holo' not in tcr_group['state'].unique().tolist() or 'apo' not in tcr_group['state'].unique().tolist():
        print('Removing group: ', group_name)
        continue
        
    tcr_group = tcr_group.copy()
    apo_holo_dfs.append(tcr_group)

apo_holo_tcrs = pd.concat(apo_holo_dfs).reset_index(drop=True)
apo_holo_tcrs

Removing group:  NSASQS-VYSSG-VVRAGKLI-MNHEY-SVGEGT-ASGQGNFDIQY


,pdb,Bchain,Achain,antigen_chain,antigen_name,mhc_chain1,mhc_chain2,docking_angle,beta_subgroup,alpha_subgroup,...,cdr_2_alpha_seq,cdr_3_alpha_seq,cdr_1_beta_seq,cdr_2_beta_seq,cdr_3_beta_seq,peptide_seq,mhc_chain_1_seq,mhc_chain_2_seq,cdr_sequence_collated,valid
0,6am5,E,D,C,ser-met-leu-gly-ile-gly-ile-val-pro-val,A,B,33.258,TRBV6,TRAV12,...,IYSNGD,AVNFGGGKLI,MRHNA,SNTAGT,ASSLSFGTEAF,SMLGIGIVPV,GSHSMRYFFTSVSRPGRGEPRFIAVGYVDDTQFVRFDSDAASQRME...,TPKIQVYSRHPAENGKSNFLNCYVSGFHPSDIEVDLLKNGERIEKV...,DRGSQS-IYSNGD-AVNFGGGKLI-MRHNA-SNTAGT-ASSLSFGTEAF,True
1,3qdj,E,D,C,mart-1(27-35) peptide,A,B,33.874,TRBV6,TRAV12,...,IYSNGD,AVNFGGGKLI,MRHNA,SNTAGT,ASSLSFGTEAF,AAGIGILTV,GSHSMRYFFTSVSRPGRGEPRFIAVGYVDDTQFVRFDSDAASQRME...,TPKIQVYSRHPAENGKSNFLNCYVSGFHPSDIEVDLLKNGERIEKV...,DRGSQS-IYSNGD-AVNFGGGKLI-MRHNA-SNTAGT-ASSLSFGTEAF,True
2,3qeu,E,D,NaN,NaN,NaN,NaN,NaN,TRBV6,TRAV12,...,IYSNGD,AVNFGGGKLI,MRHNA,SNTAGT,ASSLSFGTEAF,None,None,None,DRGSQS-IYSNGD-AVNFGGGKLI-MRHNA-SNTAGT-ASSLSFGTEAF,True
3,3qdg,E,D,C,mart-1 (26-35) peptide,A,B,34.175,TRBV6,TRAV12,...,IYSNGD,AVNFGGGKLI,MRHNA,SNTAGT,ASSLSFGTEAF,ELAGIGILTV,GSHSMRYFFTSVSRPGRGEPRFIAVGYVDDTQFVRFDSDAASQRME...,TPKIQVYSRHPAENGKSNFLNCYVSGFHPSDIEVDLLKNGERIEKV...,DRGSQS-IYSNGD-AVNFGGGKLI-MRHNA-SNTAGT-ASSLSFGTEAF,True
4,6amu,E,D,C,met-met-trp-asp-arg-gly-leu-gly-met-met,A,B,36.974,TRBV6,TRAV12,...,IYSNGD,AVNFGGGKLI,MRHNA,SNTAGT,ASSLSFGTEAF,MMWDRGLGMM,SHSMRYFFTSVSRPGRGEPRFIAVGYVDDTQFVRFDSDAASQRMEP...,TPKIQVYSRHPAENGKSNFLNCYVSGFHPSDIEVDLLKNGERIEKV...,DRGSQS-IYSNGD-AVNFGGGKLI-MRHNA-SNTAGT-ASSLSFGTEAF,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,2oi9,C,B,Q,peptide (gln)(leu)(ser)(pro)(phe)(pro)(phe)(as...,A,NaN,48.240,TRBV13,TRAV9,...,YYSGDPVV,AVSGFASALT,NNHNN,SYGAGS,ASGGGGTLY,QLSPFPFDL,GPHSMRYYETATSRRGLGEPRYTSVGYVDDKEFVRFDSDAENPRYE...,,YSATPY-YYSGDPVV-AVSGFASALT-NNHNN-SYGAGS-ASGGGGTLY,True
71,1mwa,B,A,P,dev8,H,L,22.774,TRBV13,TRAV9,...,YYSGDPVV,AVSGFASALT,NNHNN,SYGAGS,ASGGGGTLY,EQYKFYSV,GPHSLRYFVTAVSRPGLGEPRYMEVGYVDDTEFVRFDSDAENPRYE...,TPQIQVYSRHPPENGKPNILNCYVTQFHPPHIEIQMLKNGKKIPKV...,YSATPY-YYSGDPVV-AVSGFASALT-NNHNN-SYGAGS-ASGGGGTLY,True
72,1tcr,B,A,NaN,NaN,NaN,NaN,NaN,TRBV13,TRAV9,...,YYSGDPVV,AVSGFASALT,NNHNN,SYGAGS,ASGGGGTLY,None,None,None,YSATPY-YYSGDPVV-AVSGFASALT-NNHNN-SYGAGS-ASGGGGTLY,True
73,7n1e,E,D,C,spike protein s2,A,B,35.924,TRBV11,TRAV16,...,HISR,ALSGFNNAGNMLT,SGHAT,FQNNGV,ASSLGGAGGADTQY,RLQSLQTYV,GSHSMRYFFTSVSRPGRGEPRFIAVGYVDDTQFVRFDSDAASQRME...,TPKIQVYSRHPAENGKSNFLNCYVSGFHPSDIEVDLLKNGERIEKV...,YSGSPE-HISR-ALSGFNNAGNMLT-SGHAT-FQNNGV-ASSLGGA...,True


## Export

In [37]:
base_output_path = '/project/koohylab/bmcmaste/projects/tcr-loop-comparison/data/apo-holo-mhc-class-I_refined'
summary = {}

for group_name, group_data in apo_holo_tcrs.groupby('cdr_sequence_collated'):
    output_path = os.path.join(base_output_path, group_name)
    
    if not os.path.exists(output_path):
        os.mkdir(output_path)
    
    summary[group_name] = []
    
    for _, entry in group_data.iterrows():
        # isolate structure
        with open(entry['file_path_imgt'], 'r') as fh:
            structure = parse_pdb(fh.read(), silent=True).to_pandas()
        
        structure = structure.query("record_type == 'ATOM'")
        structure = structure.query("chain_id in @entry.chains")
        
        with open(os.path.join(output_path, f"{entry.pdb}_{''.join(entry.chains)}_{entry.state}.pdb"), 'w') as fh:
            fh.write(str(Structure.from_pandas(structure)))
        
        # add info to summary
        structure_summary = {'pdb_id': entry.pdb,
                              'state': entry.state,
                              'alpha_chain': entry.Achain,
                              'beta_chain': entry.Bchain}
        
        if entry.state == 'holo':
            structure_summary['peptide_chain'] = entry['antigen_chain']
            structure_summary['mhc_chain'] = entry['mhc_chain1']
        
        summary[group_name].append(structure_summary)

with open(os.path.join(base_output_path, 'summary.json'), 'w') as fh:
    json.dump(summary, fh, indent=1)

/home/b/bmcmaste/.local/lib/python3.11/site-packages/python_pdb/entities.py:509: StructureConstructionWarning: AtomRecord(atom_num=3618, atom_name='CA', alt_loc='A', res_name='ARG', chain_id='A', seq_id=14, insert_code=None, x_pos=33.789, y_pos=0.467, z_pos=85.867, occupancy=0.5, b_factor=64.23, element='C', charge=None): ATOM contains alternate location. The `split_states` method can be used to separate these possible conformations into separate models.
  warnings.warn((f'{record}: ATOM contains alternate location. '
/home/b/bmcmaste/.local/lib/python3.11/site-packages/python_pdb/entities.py:509: StructureConstructionWarning: AtomRecord(atom_num=3621, atom_name='CB', alt_loc='A', res_name='ARG', chain_id='A', seq_id=14, insert_code=None, x_pos=33.848, y_pos=-0.861, z_pos=85.051, occupancy=0.5, b_factor=64.72, element='C', charge=None): ATOM contains alternate location. The `split_states` method can be used to separate these possible conformations into separate models.
  warnings.warn(

/home/b/bmcmaste/.local/lib/python3.11/site-packages/python_pdb/entities.py:509: StructureConstructionWarning: AtomRecord(atom_num=163, atom_name='N', alt_loc='A', res_name='THR', chain_id='E', seq_id=13, insert_code=None, x_pos=19.416, y_pos=-15.345, z_pos=17.709, occupancy=0.5, b_factor=58.38, element='N', charge=None): ATOM contains alternate location. The `split_states` method can be used to separate these possible conformations into separate models.
  warnings.warn((f'{record}: ATOM contains alternate location. '
/home/b/bmcmaste/.local/lib/python3.11/site-packages/python_pdb/entities.py:509: StructureConstructionWarning: AtomRecord(atom_num=164, atom_name='CA', alt_loc='A', res_name='THR', chain_id='E', seq_id=13, insert_code=None, x_pos=20.487, y_pos=-16.349, z_pos=17.732, occupancy=0.5, b_factor=58.45, element='C', charge=None): ATOM contains alternate location. The `split_states` method can be used to separate these possible conformations into separate models.
  warnings.warn(

/home/b/bmcmaste/.local/lib/python3.11/site-packages/python_pdb/entities.py:509: StructureConstructionWarning: AtomRecord(atom_num=1334, atom_name='N', alt_loc='A', res_name='CYS', chain_id='B', seq_id=184, insert_code=None, x_pos=28.455, y_pos=3.985, z_pos=20.385, occupancy=0.5, b_factor=35.76, element='N', charge=None): ATOM contains alternate location. The `split_states` method can be used to separate these possible conformations into separate models.
  warnings.warn((f'{record}: ATOM contains alternate location. '
/home/b/bmcmaste/.local/lib/python3.11/site-packages/python_pdb/entities.py:509: StructureConstructionWarning: AtomRecord(atom_num=1335, atom_name='CA', alt_loc='A', res_name='CYS', chain_id='B', seq_id=184, insert_code=None, x_pos=29.636, y_pos=4.049, z_pos=21.214, occupancy=0.5, b_factor=37.38, element='C', charge=None): ATOM contains alternate location. The `split_states` method can be used to separate these possible conformations into separate models.
  warnings.warn(

/home/b/bmcmaste/.local/lib/python3.11/site-packages/python_pdb/entities.py:509: StructureConstructionWarning: AtomRecord(atom_num=3814, atom_name='CB', alt_loc='A', res_name='ASP', chain_id='A', seq_id=30, insert_code=None, x_pos=40.776, y_pos=-12.987, z_pos=20.572, occupancy=0.5, b_factor=53.07, element='C', charge=None): ATOM contains alternate location. The `split_states` method can be used to separate these possible conformations into separate models.
  warnings.warn((f'{record}: ATOM contains alternate location. '
/home/b/bmcmaste/.local/lib/python3.11/site-packages/python_pdb/entities.py:509: StructureConstructionWarning: AtomRecord(atom_num=3815, atom_name='CG', alt_loc='A', res_name='ASP', chain_id='A', seq_id=30, insert_code=None, x_pos=40.051, y_pos=-11.67, z_pos=20.833, occupancy=0.5, b_factor=52.11, element='C', charge=None): ATOM contains alternate location. The `split_states` method can be used to separate these possible conformations into separate models.
  warnings.war

/home/b/bmcmaste/.local/lib/python3.11/site-packages/python_pdb/entities.py:509: StructureConstructionWarning: AtomRecord(atom_num=3497, atom_name='N', alt_loc='A', res_name='ASN', chain_id='A', seq_id=207, insert_code=None, x_pos=-8.451, y_pos=-13.485, z_pos=40.599, occupancy=0.5, b_factor=41.38, element='N', charge=None): ATOM contains alternate location. The `split_states` method can be used to separate these possible conformations into separate models.
  warnings.warn((f'{record}: ATOM contains alternate location. '
/home/b/bmcmaste/.local/lib/python3.11/site-packages/python_pdb/entities.py:509: StructureConstructionWarning: AtomRecord(atom_num=3498, atom_name='CA', alt_loc='A', res_name='ASN', chain_id='A', seq_id=207, insert_code=None, x_pos=-9.575, y_pos=-12.867, z_pos=39.893, occupancy=0.5, b_factor=39.65, element='C', charge=None): ATOM contains alternate location. The `split_states` method can be used to separate these possible conformations into separate models.
  warnings.w

...

/home/b/bmcmaste/.local/lib/python3.11/site-packages/python_pdb/entities.py:509: StructureConstructionWarning: AtomRecord(atom_num=350, atom_name='N', alt_loc='A', res_name='ARG', chain_id='E', seq_id=43, insert_code=None, x_pos=26.51, y_pos=1.849, z_pos=30.322, occupancy=0.5, b_factor=21.32, element='N', charge=None): ATOM contains alternate location. The `split_states` method can be used to separate these possible conformations into separate models.
  warnings.warn((f'{record}: ATOM contains alternate location. '
/home/b/bmcmaste/.local/lib/python3.11/site-packages/python_pdb/entities.py:509: StructureConstructionWarning: AtomRecord(atom_num=351, atom_name='CA', alt_loc='A', res_name='ARG', chain_id='E', seq_id=43, insert_code=None, x_pos=25.099, y_pos=2.061, z_pos=30.572, occupancy=0.5, b_factor=19.67, element='C', charge=None): ATOM contains alternate location. The `split_states` method can be used to separate these possible conformations into separate models.
  warnings.warn((f'{r

/home/b/bmcmaste/.local/lib/python3.11/site-packages/python_pdb/entities.py:509: StructureConstructionWarning: AtomRecord(atom_num=1262, atom_name='N', alt_loc='A', res_name='HIS', chain_id='E', seq_id=167, insert_code=None, x_pos=12.272, y_pos=13.171, z_pos=36.266, occupancy=0.5, b_factor=42.36, element='N', charge=None): ATOM contains alternate location. The `split_states` method can be used to separate these possible conformations into separate models.
  warnings.warn((f'{record}: ATOM contains alternate location. '
/home/b/bmcmaste/.local/lib/python3.11/site-packages/python_pdb/entities.py:509: StructureConstructionWarning: AtomRecord(atom_num=1263, atom_name='CA', alt_loc='A', res_name='HIS', chain_id='E', seq_id=167, insert_code=None, x_pos=13.447, y_pos=13.934, z_pos=36.436, occupancy=0.5, b_factor=43.42, element='C', charge=None): ATOM contains alternate location. The `split_states` method can be used to separate these possible conformations into separate models.
  warnings.war

/home/b/bmcmaste/.local/lib/python3.11/site-packages/python_pdb/entities.py:509: StructureConstructionWarning: AtomRecord(atom_num=3739, atom_name='CA', alt_loc='A', res_name='ARG', chain_id='A', seq_id=6, insert_code=None, x_pos=-43.021, y_pos=-4.416, z_pos=25.234, occupancy=0.5, b_factor=29.92, element='C', charge=None): ATOM contains alternate location. The `split_states` method can be used to separate these possible conformations into separate models.
  warnings.warn((f'{record}: ATOM contains alternate location. '
/home/b/bmcmaste/.local/lib/python3.11/site-packages/python_pdb/entities.py:509: StructureConstructionWarning: AtomRecord(atom_num=3742, atom_name='CB', alt_loc='A', res_name='ARG', chain_id='A', seq_id=6, insert_code=None, x_pos=-44.411, y_pos=-3.841, z_pos=24.954, occupancy=0.5, b_factor=33.66, element='C', charge=None): ATOM contains alternate location. The `split_states` method can be used to separate these possible conformations into separate models.
  warnings.warn

/home/b/bmcmaste/.local/lib/python3.11/site-packages/python_pdb/entities.py:509: StructureConstructionWarning: AtomRecord(atom_num=345, atom_name='N', alt_loc='A', res_name='ARG', chain_id='E', seq_id=22, insert_code=None, x_pos=9.223, y_pos=-10.732, z_pos=60.575, occupancy=0.5, b_factor=23.54, element='N', charge=None): ATOM contains alternate location. The `split_states` method can be used to separate these possible conformations into separate models.
  warnings.warn((f'{record}: ATOM contains alternate location. '
/home/b/bmcmaste/.local/lib/python3.11/site-packages/python_pdb/entities.py:509: StructureConstructionWarning: AtomRecord(atom_num=346, atom_name='CA', alt_loc='A', res_name='ARG', chain_id='E', seq_id=22, insert_code=None, x_pos=10.252, y_pos=-11.485, z_pos=61.277, occupancy=0.5, b_factor=24.38, element='C', charge=None): ATOM contains alternate location. The `split_states` method can be used to separate these possible conformations into separate models.
  warnings.warn((

/home/b/bmcmaste/.local/lib/python3.11/site-packages/python_pdb/entities.py:509: StructureConstructionWarning: AtomRecord(atom_num=7933, atom_name='N', alt_loc='B', res_name='ARG', chain_id='A', seq_id=82, insert_code=None, x_pos=24.065, y_pos=-5.876, z_pos=-15.623, occupancy=0.51, b_factor=32.65, element='N', charge=None): ATOM contains alternate location. The `split_states` method can be used to separate these possible conformations into separate models.
  warnings.warn((f'{record}: ATOM contains alternate location. '
/home/b/bmcmaste/.local/lib/python3.11/site-packages/python_pdb/entities.py:509: StructureConstructionWarning: AtomRecord(atom_num=7934, atom_name='CA', alt_loc='B', res_name='ARG', chain_id='A', seq_id=82, insert_code=None, x_pos=24.425, y_pos=-7.276, z_pos=-15.807, occupancy=0.51, b_factor=36.07, element='C', charge=None): ATOM contains alternate location. The `split_states` method can be used to separate these possible conformations into separate models.
  warnings.w